In [1]:
# library
import numpy as np
import pandas as pd
import plotly.express as px

import json

site_list = ["bbc", "cnn", "foxnews", "nationalreview"]

In [2]:
# open json file
def open_json(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    # return data 

    # file_path = "data/" + site_list[0] + "/articles.json"

    json_data = open_json(file_path)
    data = [{"url": url, **details} for url, details in json_data.items() if details is not None]
    df = pd.DataFrame(data)

    df_selected = df[["title", "date_publish", "description", "maintext", "authors", "wayback_id"]]
    # df_selected.describe()
    return df_selected

In [3]:
def cleanDF(df_selected):
    # clean the data
    # drop the main text that is empty
    df_selected = df_selected.dropna(subset=["maintext"])

    # drop the duplicates
    df_selected = df_selected.drop_duplicates(subset=["maintext"])

    # add a new column based on the wayback_id
    df_selected["wayback_time"] = pd.to_datetime((df_selected["wayback_id"]).astype(str).str[:8], format='%Y%m%d', errors='coerce')

    # add a new column based on the length of the main text
    df_selected["text_len"] = df_selected["maintext"].apply(lambda x: len(x))

    df_selected.drop(columns=["wayback_id"], inplace=True)

    # sort
    df_selected = df_selected.sort_values(by="wayback_time", ascending=True)

    # print(df_selected.describe())
    # print(df_selected.head())
    # df_selected.head()
    return df_selected

In [4]:
# filter the data back on title 

In [5]:
def graphDF(df_selected):
    df_selected['year_week'] = df_selected['wayback_time'].dt.strftime('%Y-%U')
    df_selected['week_start_date'] = pd.to_datetime(df_selected['year_week'] + '-0', format='%Y-%U-%w')

    # sort the dataframe by date
    df_selected_weekly = df_selected.groupby('week_start_date')['title'].count().reset_index()
    df_selected_year_week = df_selected.groupby('year_week').count().reset_index()
    df_selected_year_week.sort_values('year_week', inplace=True)

    # plot the number of urls per day
    fig = px.line(
        df_selected_weekly, 
        x='week_start_date', 
        y='title',
        labels={'week_start_date': 'Week Starting', 'title': 'Number of Articles'},
        template='seaborn'
    )

    fig.update_layout(
        xaxis_title='Week Starting',
        yaxis_title='Number of Articles',
        title='Number of Articles per Week',
        title_x=0.5
    )

    fig.show()


In [6]:
for site in site_list[:1]:
    file_path = "data/" + site + "/articles.json"
    df_selected = open_json(file_path)
    df_selected = cleanDF(df_selected)
    graphDF(df_selected)
    print("done")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x11819bfb0>>
Traceback (most recent call last):
  File "/Users/weikuo/Documents/github-repositories/web-scrape-newswebsites/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
